# 利用Qwen 翻译题目

In [1]:
import os
import time

import pandas as pd
from openai import OpenAI
import pandas

In [2]:
import httpx

language_list = ["python"]
suffix_map = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "java": "java",
    "c&cpp": "cpp"
}
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-ff167cad89364e2491fc6193e006c4a3",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

In [3]:
prompt_message_template = {
    # py 2 js
    "python_answer_to_javascript": "You are a highly skilled developer and programming expert proficient in both Python and JavaScript. Your task is to convert the following Python code into its equivalent JavaScript code, ensuring that the functionality, logic, and structure remain consistent. Here is the Python code: {python_code}",
    "python_signature_to_javascript": "You are an expert in both Python and JavaScript programming. Your task is to convert Python function signatures or class signatures into equivalent JavaScript syntax, while preserving the purpose and structure of the code. Additionally, you must ensure that Python comments are rewritten to conform to JavaScript documentation standards using JSDoc format. Here is the Python code with its function/class signature and comments: {python_code}",
    "python_test_to_javascript": "You are an expert in Python's unittest framework and JavaScript's jest testing framework. Your task is to convert the given Python unittest test class into an equivalent test class using jest. Make sure that the functionality and logic of the tests are maintained. Any necessary changes due to language differences between Python and JavaScript should be documented in the conversion. Additionally, ensure that the structure is clear and that all assertions in unittest are mapped to appropriate jest equivalents. Here is the Python code: {python_code}",
    # py 2 ts
    "python_answer_to_typescript": "You are an expert in Python and TypeScript programming. Your task is to convert the provided Python code into equivalent TypeScript code. Ensure that the functionality, structure, and logic remain consistent, while also adjusting for any necessary language differences. You should properly define types in TypeScript, following TypeScript's static type system. Additionally, any Python-specific features should be adapted to idiomatic TypeScript where needed. Here is the Python code: {python_code}",
    "python_signature_to_typescript": "You are proficient in both Python and TypeScript programming. Your task is to convert Python function signatures or class signatures into equivalent TypeScript code, while preserving the original functionality and structure. In addition, convert Python comments and docstrings to appropriate TypeScript documentation comments using the /** */ JSDoc style. Ensure that type annotations in Python are properly translated to TypeScript types, and make necessary adjustments for any language differences. Here is the Python code: {python_code}",
    "python_test_to_typescript": "You are proficient in both Python's unittest framework and TypeScript's jest testing framework. Your task is to convert the provided Python unittest test class into an equivalent test class using jest in TypeScript. Make sure that the functionality and logic of the tests are preserved. Adjust the code to account for the differences between Python and TypeScript, and ensure that the necessary type annotations are included in the TypeScript code. The assert methods in unittest should be mapped to the appropriate jest assertions. Here is the Python code: {python_code}",
    # py 2 c&cpp
    "python_answer_to_c&cpp": "You are an expert in both Python and C++ programming. Your task is to convert the provided Python code into equivalent C++ code. Ensure that the functionality, logic, and structure remain consistent while adapting to the syntax and features of C++. Make sure to handle any language-specific features, such as type declarations, memory management, or loops, and translate them into their appropriate C++ counterparts. Here is the Python code: {python_code}",
    "python_signature_to_c&cpp": "You are proficient in both Python and C++ programming. Your task is to convert Python function signatures or class signatures into their equivalent C++ versions, while preserving the original purpose and structure of the code. You must also convert Python comments and docstrings into appropriate C++ comments using // for single-line comments and /* */ for multi-line comments, adhering to C++ documentation standards. Additionally, ensure that Python type annotations are correctly translated into C++ types. Here is the Python code: {python_code}",
    "python_test_to_c&cpp": "You are proficient in both Python's unittest framework and C++'s Catch2 testing framework. Your task is to convert the provided Python unittest test class into an equivalent test class using Catch2 in C++. Ensure that the functionality and logic of the tests are preserved. Make any necessary adjustments for language differences between Python and C++, and use appropriate assertions in Catch2. Here is the Python code: {python_code}",
    # py 2 java
    "python_answer_to_java": "You are an expert in both Python and Java programming. Your task is to convert the provided Python code into equivalent Java code. The Java code should be organized with the package name org.real.temp and the class name Answer. Ensure that the functionality, logic, and structure remain consistent while adapting to Java's syntax and language features. Make sure to handle type declarations, method signatures, and object-oriented principles in Java appropriately. Here is the Python code:  {python_code}",
    "python_signature_to_java": "You are proficient in both Python and Java programming. Your task is to convert Python function signatures or class signatures into their equivalent Java versions, while preserving the original functionality and structure. Additionally, you must convert Python comments and docstrings into appropriate Java-style comments using the /** */ Javadoc format. Make sure that Python type annotations are translated into appropriate Java types, and that the method or class is structured according to Java's conventions. Here is the Python code: {python_code}",
    "python_test_to_java": "You are proficient in both Python's unittest framework and Java's JUnit testing framework. Your task is to convert the provided Python unittest test class into an equivalent test class using JUnit in Java. The Java test class should be placed in the package org.real.temp and named Tester. Ensure that the functionality and logic of the tests are preserved, and use appropriate JUnit annotations (e.g., @Test) and assertion methods. Make necessary adjustments for differences between Python and Java syntax and structure. Here is the Python code: {python_code}",

    # js 2 py
    "javascript_answer_to_python": "You are an expert in both JavaScript and Python programming. Your task is to convert the provided JavaScript code into equivalent Python code. Ensure that the functionality, logic, and structure remain consistent, while adapting the syntax and language-specific features of JavaScript to their appropriate Python counterparts. Handle any necessary changes in variable declaration, loops, and functions, and ensure the Python version maintains the same behavior as the original JavaScript code. Here is the JavaScript code: {javascript_code}",
    "javascript_signature_to_python":"You are proficient in both JavaScript and Python programming. Your task is to convert JavaScript function signatures or class signatures into equivalent Python versions, while preserving the original functionality and structure. Additionally, you must convert JavaScript comments (both single-line // and multi-line /* */) into appropriate Python-style comments (using # for single-line comments and \"\"\" for multi-line docstrings). Make sure to adapt JavaScript-specific syntax and conventions to Pythonic equivalents. Here is the JavaScript code: {javascript_code}",
    "javascript_test_to_python": "please change this javascript jest testcase to python testcase use unittest, the javascript jest testcase code as below {javascript_code}",
    # js 2 ts
    "javascript_answer_to_typescript": "please change this javascript code to typescript code,the javascript code as below {javascript_code}",
    "javascript_signature_to_typescript": "please change this javascript code signature to typescript code signature,keep the code comments, the javascript code signature as below {javascript_code}",
    "javascript_test_to_typescript": "please change this javascript jest testcase to typescript testcase use jest describe testcase, the javascript jest testcase code as below {javascript_code}",
    # js 2 c&cpp
    "javascript_answer_to_c&cpp": "please change this javascript code to cpp code,the javascript code as below {javascript_code}",
    "javascript_signature_to_c&cpp": "please change this javascript code signature to cpp code signature,keep the code comments, the javascript code signature as below {javascript_code}",
    "javascript_test_to_c&cpp": "please change this javascript jest testcase to cpp testcase use catch2 testcase,the javascript jest testcase code as below {javascript_code}",
    # js 2 java
    "javascript_answer_to_java": "please change this javascript code to java code,the javascript code as below {javascript_code}",
    "javascript_signature_to_java": "please change this javascript code signature to java code signature,keep the code comments, the javascript code signature as below {javascript_code}",
    "javascript_test_to_java": "please change this javascript jest testcase to java testcase use junit,package is org.real.temp,class is Tester,the javascript jest testcode as below {javascript_code}",



    # ts 2 py
    "typescript_answer_to_python": "please change this typescript code to python code,the typescript code as below {typescript_code}",
    "typescript_signature_to_python": "please change this typescript code signature to python code signature,keep the code comments, the typescript code signature as below {typescript_code}",
    "typescript_test_to_python": "please change this typescript jest testcase to python testcase use unittest,the typescript jest testcase code as below {typescript_code}",

    # ts 2 js
    "typescript_answer_to_javascript": "please change this typescript code to javascript code,the typescript code as below {typescript_code}",
    "typescript_signature_to_javascript": "please change this typescript code signature to javascript code signature,keep the code comments, the typescript code signature as below {typescript_code}",
    "typescript_test_to_javascript": "please change this typescript jest testcase to javascript testcase use jest describe testcase, the typescript jest testcase code as below {typescript_code}",

    # ts 2 c&cpp
    "typescript_answer_to_c&cpp": "please change this typescript code to cpp code,the typescript code as below {typescript_code}",
    "typescript_signature_to_c&cpp": "please change this typescript code signature to cpp code signature,keep the code comments, the typescript code signature as below {typescript_code}",
    "typescript_test_to_c&cpp": "please change this typescript jest testcase to cpp testcase use catch2 testcase,the typescript jest testcase code as below {typescript_code}",

    # ts 2 java
    "typescript_answer_to_java": "please change this typescript code to java code,the typescript code as below {typescript_code}",
    "typescript_signature_to_java": "please change this typescript code signature to java code signature,keep the code comments, the typescript code signature as below {typescript_code}",
    "typescript_test_to_java": "please change this typescript jest testcase to java testcase use junit,package is org.real.temp,class is Tester,the javascript jest testcode as below {typescript_code}",

}

In [ ]:
from pathlib import Path

for language in language_list:
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    l_list = [x for x in ["python", "javascript", "typescript", "c&cpp", "java"] if x != language]
    for index, row in data.iterrows():
        task_id = row['task_id']
        if task_id in [i for i in range(1, 434)]:
            continue
        work_path = f"../all/t{task_id}"
        # 读取signature代码
        signature_path = f"{work_path}/{language}/signature.{suffix_map[language]}"
        with open(signature_path, "r", encoding="utf8") as file:
            signature_code = file.read()
        # 读取test代码
        test_path = f"{work_path}/{language}/test.{suffix_map[language]}"
        with open(signature_path, "r", encoding="utf8") as file:
            test_code = file.read()
        # 读取answer代码
        answer_path = f"{work_path}/{language}/answer.{suffix_map[language]}"
        with open(answer_path, "r", encoding="utf8") as file:
            answer_code = file.read()
        for l in l_list:
            # 创建文件夹
            if l != "java":
                Path(f"{work_path}/{l}").mkdir(parents=True, exist_ok=True)
                answer_file_path = Path(f"{work_path}/{l}/answer.{suffix_map[l]}")
                signature_file_path = Path(f"{work_path}/{l}/signature.{suffix_map[l]}")
                test_file_path = Path(f"{work_path}/{l}/test.{suffix_map[l]}")
                if not answer_file_path.exists():
                    answer_file_path.touch()
                if not signature_file_path.exists():
                    signature_file_path.touch()
                if not test_file_path.exists():
                    test_file_path.touch()
            elif l == "java":
                Path(f"{work_path}/{l}").mkdir(parents=True, exist_ok=True)
                answer_file_path = Path(f"{work_path}/{l}/Answer.{suffix_map[l]}")
                signature_file_path = Path(f"{work_path}/{l}/Signature.{suffix_map[l]}")
                test_file_path = Path(f"{work_path}/{l}/Tester.{suffix_map[l]}")
                if not answer_file_path.exists():
                    answer_file_path.touch()
                if not signature_file_path.exists():
                    signature_file_path.touch()
                if not test_file_path.exists():
                    test_file_path.touch()
            print(f"{language}_{task_id}_to_{l} ===============================\n")
            # 答案转换 =============================
            message_list = []
            answer_key = f"{language}_answer_to_{l}"
            answer_template = prompt_message_template[answer_key].format(python_code=answer_code)
            message_list.append({'role': 'user', 'content': answer_template})
            completion = client.chat.completions.create(
                model="qwen-plus",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                messages=message_list,
            )
            llm_answer_response = completion.choices[0].message.content

            message_list.append({'role': 'assistant', 'content': llm_answer_response})
            with open(f"{work_path}/{l}/answer.txt", "w", encoding="utf8") as file:
                file.write(llm_answer_response)
                file.flush()

            # 签名转换=============================
            signature_key = f"{language}_signature_to_{l}"
            signature_template = prompt_message_template[signature_key].format(python_code=signature_code)
            message_list.append({'role': 'user', 'content': signature_template})
            completion = client.chat.completions.create(
                model="qwen-plus",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                messages=message_list,
            )
            llm_signature_response = completion.choices[0].message.content
            message_list.append({'role': 'assistant', 'content': llm_signature_response})
            with open(f"{work_path}/{l}/signature.txt", "w", encoding="utf8") as file:
                file.write(llm_signature_response)
                file.flush()

            # 测试转换=============================
            test_key = f"{language}_test_to_{l}"
            test_template = prompt_message_template[test_key].format(python_code=test_code)
            message_list.append({'role': 'user', 'content': test_template})
            completion = client.chat.completions.create(
                model="qwen-plus",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                messages=message_list,
            )
            llm_test_response = completion.choices[0].message.content
            with open(f"{work_path}/{l}/test.txt", "w", encoding="utf8") as file:
                file.write(llm_test_response)
                file.flush()
            time.sleep(0.5)


python_438_to_javascript ===============================

python_438_to_typescript ===============================

python_438_to_c&cpp ===============================

python_438_to_java ===============================

python_442_to_javascript ===============================

python_442_to_typescript ===============================

